In [1]:
import glacierml

2023-01-12 15:02:41.659011: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.


# Step 0. Match the indices in RGI with those in GlaThiDa
This corresponds to the file ```workflow_step0_merge_training_data.py```. But note that this is can't really be run in a notebook because with ```useMP=True``` then note that multiprocessing doesn't play well with notebooks...

In [2]:
pth = '/data/fast1/glacierml/data/'
# output=glacierml.match_GlaThiDa_RGI_index(pth=pth, useMP=True);
# This writes the file to:  pth + '/v2/GlaThiDa_with_RGIId_v2.csv'

# Step 1. Build and train models.
This corresponds to the file ```workflow_step1_run_bootstrap_and_ensemble.py```.
Here, to illustrate the general ideas, we just build and train a single model rather than
going through the bootstrap/ensemble process.

First, load the training data,

In [3]:
parameterization, dataset, dataset.name, res = \
    glacierml.select_dataset_coregistration(pth=pth)
dataset

# of raw thicknesses: 500
# of raw thickness matched to RGI = 500, 446 unique


,CenLat,CenLon,Slope,Zmin,Zmed,Zmax,Area,Aspect,Lmax,Thickness
0,63.469173,-146.528168,12.0,722,1841,3081,234.583,30,41911,235.0
1,63.281000,-145.427000,14.0,1162,1858,2438,17.567,172,8639,147.0
2,58.380000,-134.349000,10.0,676,1123,1494,9.528,327,6332,66.0
9,51.177000,-116.330000,15.3,2390,2526,2853,0.872,69,1181,70.0
11,44.148300,-121.773320,20.7,2321,2590,2810,0.808,91,1680,26.0
...,...,...,...,...,...,...,...,...,...,...
483,-33.403600,-69.822000,10.0,4255,5492,5885,7.233,298,8109,122.0
485,-33.222300,-70.205600,21.1,3469,4970,5364,5.374,88,4739,38.0
489,-33.029600,-70.098400,29.0,2929,5121,5894,8.376,358,8560,59.0
492,-28.381400,-69.609100,11.6,5068,5406,5753,5.209,79,3782,67.0


Then train a single NN model with the data:

In [4]:
lr_input = 0.01
rs = 0
ep_input = 2000
layer_1_input = 10
layer_2_input = 5
dropout = True

dnn_model = glacierml.build_and_train_model(
                        dataset, 
                        learning_rate = float(lr_input), 
                        random_state = rs, 
                        epochs = int(ep_input), 
                        parameterization = parameterization, 
                        res = res,
                        layer_1 = layer_1_input,
                        layer_2 = layer_2_input,
                        dropout = dropout,
                        writeToFile = False
)   

2023-01-12 15:02:44.767708: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudnn.so.8'; dlerror: libcudnn.so.8: cannot open shared object file: No such file or directory
2023-01-12 15:02:44.767729: W tensorflow/core/common_runtime/gpu/gpu_device.cc:1850] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...
2023-01-12 15:02:44.768518: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [5]:
dnn_model